In [1]:
import os
import csv, json, pickle
import time
from collections import defaultdict, Counter
from itertools import combinations, product, chain

import numpy as np
import scipy.stats as stats
import statsmodels.stats.multitest as ssm
import graph_tool.all as gt

%matplotlib inline 
import matplotlib.pyplot as plt
import matplotlib.gridspec as gs
from matplotlib.colors import LinearSegmentedColormap, to_rgb, to_rgba
from matplotlib.ticker import AutoMinorLocator
import seaborn as sns
sns.set_style('white')

plt.rcParams['font.size'] = 12
plt.rcParams['font.family'] = "Calibri"
plt.rcParams['svg.fonttype'] = 'none'

blue = "#007EEA"
orange = "#FEB80A"
green = "#7FD13B"
pink = "#EA157A"
white = "#FFFFFF"
purple = "#754AB2"
greyRed = LinearSegmentedColormap.from_list('greyRed', ['#DCDCDC', 'w', "#C41E3A"])
whiteRed = LinearSegmentedColormap.from_list('whiteRed', ['w', 'w', '#C41E3A'])
whitePink = LinearSegmentedColormap.from_list("whitePink", ['w', 'w', pink])

#defaultBackend = 'module://ipykernel.pylab.backend_inline'

#from tqdm import tqdm_notebook as tqdm

In [2]:
default_node, default_edge = "#DCDCDC", "#696969"

#domain_order = {d:idx for idx,d in enumerate(["SH2", "PTB", "Kinase_TK","PTP", "WH1", "WW", "SH3", "PDZ"])}
domain_order = ["SH2", "PTB", "Kinase_TK","PTP", "WH1", "WW", "SH3", "PDZ"]
domain_colors = ["#A6D2F8","#59ABF1", "#007EEA", "#0D5BA1", "#CCEDB1", "#7FD13B", "#5E9031", orange, default_node]

peptide_colors = [blue, green, orange, default_node]
peptide_order = ["phosphosite", "polyproline", "c-terminus"] #["Phosphosite", "Poly-Proline", "C-Terminus"]

# Graph construction and plotting code

In [20]:
def plot_graph(graph, included_neighborhood=None, min_likelihood=None, max_neighborhood=None, find_shared=None,
               peptide_pichart=False, domain_pichart=False, 
               peptide_colors=peptide_colors, domain_colors=domain_colors,
               graph_size=(3600,3600), output_fname=None, seed=None,
               vertex_size=None, edge_width=1, edge_color=default_edge, adjust_aspect=True, layout=None):
    """
    Plots output graphs 
    
    Inputs:
        graph: graph_tool graph input with properties as defined in the data loading section. 
        included_neighborhood: a set of vertices to include. Can be used to pass in a specific subset of nodes to plot.
        min_likelihood: minimum likelihood to plot (note, invert estimated FDR's)
        max_neighborhood: largest number of edges to include around 
        find_shared: plots a graph showing the graph intersection based on a passed in set of nodes.
            Can be helpful for examining a subset of domain nodes. 
        peptide_pichart, domain_pichart: arguments for plotting either peptide or domain picharts
        peptide_colors, domain_colors: colors assigned for plotting either peptide or domain colors
        graph_size: plotting option for figure size. 
        output_fname: output file name to save graph to.         
        vertex_size: option for plotting vertex sizes
        edge_width, edge_color: options for controlling edge width and color.
        
    """
    
    np.random.seed(seed)
    if seed is not None: gt.seed_rng(seed)
    
    # Initial global filter on edges on the basis of 
    min_likelihood = 0 if min_likelihood is None else min_likelihood
    gview = gt.GraphView(graph, efilt=lambda e: graph.ep.likelihood[e] >= min_likelihood)

    def edge_filter(gview, filt_set, neighborhood_thr=None):
        """
        Helper function for creating a new graph view associated with a given filtering set.
        
        Inputs:
            gview: a prior graph to update with the new filter.
            filt_set: a set of vertices to filter down to.
            neighborhood_thr: a maximum neighborhood size to filter to. 
        """
        included_edges = set()
        for v in filter(lambda vmem: graph.vp.name[vmem] in filt_set, gview.vertices()):
            incident_edges = [(e, graph.ep.likelihood[e]) for e in v.all_edges()]
            
            if neighborhood_thr is not None:
                incident_edges = sorted(incident_edges, key=lambda x: x[1])
                included_edges |= set(etup[0] for etup in incident_edges[-neighborhood_thr:])
            else:
                included_edges |= set(etup[0] for etup in incident_edges)
    
        gview = gt.GraphView(gview, efilt=lambda e: e in included_edges)
        return gview

    # If a specific subset of nodes is desired, 
    if included_neighborhood:
        gview = edge_filter(gview, included_neighborhood, neighborhood_thr=max_neighborhood)
    elif max_neighborhood:
        domain_containing_vertices = set(gview.vp['name'][v] for v in gview.vertices() if np.sum(gview.vp['domain_dist'][v][:-1]) > 1e-5)
        gview = edge_filter(gview, domain_containing_vertices, neighborhood_thr=max_neighborhood)

        
    if find_shared is not None:
        intersection = list()
        for s in find_shared:
            adj = set(n for v in gview.vertices() for n in v.all_neighbors() if gview.vp.name[v] in s)
            intersection.append(adj)
        intersection = set.intersection(*intersection)
        
        base = set.intersection(*find_shared)
        intersection |= set(v for v in gview.vertices() if gview.vp.name[v] in base)
        
        gview = gt.GraphView(gview, vfilt=lambda v: v in intersection)
    
    gview = gt.GraphView(gview, vfilt=lambda v: v.out_degree() > 0)
    
    vertex_spec = dict()
    if vertex_size: vertex_spec['size'] = vertex_size
        
    edge_spec = {
        "color": edge_color,
        "pen_width": edge_width
    }
    
    if layout is None: layout = gt.sfdp_layout(gview)
    if peptide_pichart or domain_pichart:
        if peptide_pichart and domain_pichart: raise ValueError("Passed in conflicting options - domain and peptide pi chart")
        
        vertex_spec["shape"] = "pie"
        if peptide_pichart:
            vertex_spec["pie_fractions"] = graph.vp.peptide_dist
            vertex_spec["pie_colors"] = peptide_colors
            gt.graph_draw(gview, pos=layout, vprops=vertex_spec, eprops=edge_spec, output_size=graph_size,
                          output=output_fname, adjust_aspect=adjust_aspect)
        else:
            vertex_spec["pie_fractions"] = graph.vp.domain_dist
            vertex_spec["pie_colors"] = domain_colors
            gt.graph_draw(gview, pos=layout, vprops=vertex_spec, eprops=edge_spec, output_size=graph_size,
                          output=output_fname, adjust_aspect=adjust_aspect)
        
    else:
        vertex_spec["fill_color"] = default_node
        gt.graph_draw(gview, pos=layout, vprops=vertex_spec, eprops=edge_spec, output_size=graph_size,
                      output=output_fname, adjust_aspect=adjust_aspect)
    return gview, layout

# Graph data Loading

In [4]:
def load_graph(input_edges, domain_metadata, peptide_metadata, dorder=domain_order, porder=peptide_order):
    """
    Creates a graph_tool graph with 
    
    Inputs:
        input_edges
        domain_metadata
        peptide_metadata
    """
    
    g = gt.Graph(directed=False)
    edge_weights = g.new_edge_property("float")
    node_props = g.add_edge_list(input_edges, hashed=True, hash_type="string", eprops=[edge_weights])
    g.vertex_properties['name'] = node_props
    g.edge_properties['likelihood'] = edge_weights
    
    def create_metadata_property(metadata_dct, metadata_order):
        new_property = g.new_vertex_property("vector<float>")
        for vertex in g.vertices():
            new_distribution = np.zeros(len(metadata_order) + 1).astype(np.float64)
            vertex_metadata = metadata_dct.get(g.vp.name[vertex], list())
            
            if len(vertex_metadata) == 0:
                new_distribution[-1] = 1
            else:
                cts = Counter(vertex_metadata)
                for idx, m in enumerate(metadata_order):
                    new_distribution[idx] = cts.get(m, 0)
            
            new_distribution /= np.sum(new_distribution)
            new_property[vertex] = new_distribution
        
        return new_property
    
    g.vertex_properties['domain_dist'] = create_metadata_property(domain_metadata, dorder)
    g.vertex_properties['peptide_dist'] = create_metadata_property(peptide_metadata, porder)
          
    return g

# Human PPI network

Load HSM/P predictions. p-values (combined HSM/D p-values) computed previously for comparison to experiment.

In [5]:
import pandas as pd
df_ppis = pd.read_csv('data/perf/hsmp/ppi_predictions_pvals.csv')
ppis = df_ppis.to_numpy()

In [6]:
from scripts.utils import load_protein_metadata, get_protein_composition
metadata = load_protein_metadata()

In [7]:
# get all domain data and peptide data for each protein among the predicted PPIs
domain_data = defaultdict(list)
peptide_data = defaultdict(list)
prots = np.unique(ppis[:,0:1].flatten())
for prot in prots:
    c = get_protein_composition(prot, metadata=metadata)
    dlst = [d[0] for d in c if d[0] in domain_order]
    domain_data[prot] = dlst #if len(dlst) > 0 else ["None"]
    plst = [p[0] for p in c if p[0] in peptide_order]
    peptide_data[prot] = plst #if len(plst) > 0 else ["none"]

## Figure 6: Network of PPIs predicted at FDR of 0.01

In [25]:
reject, _, _, _ = ssm.multipletests(ppis[:,3], alpha=0.01, method='fdr_bh')
ppis_fdr = ppis[reject]
edges_fdr = [[str(p[0]), str(p[1]), 1.0-float(p[3])] for p in ppis_fdr]
print(len(ppis_fdr))

graph_fdr = load_graph(edges_fdr, domain_data, peptide_data)

143215


In [29]:
output_dir = "plots/fig_6_supp_fig_7"
if not os.path.exists(output_dir): os.makedirs(output_dir)

# Fig 6a: Nodes colored by domain composition
_, layout = plot_graph(graph_fdr, domain_pichart=True, max_neighborhood=50, seed=111,
           graph_size=(200,200), edge_width=0.01, vertex_size=1.5, edge_color="#000000",
           output_fname=os.path.join(output_dir,'ppi_network_fdr0.01.domain_color.max_adj50.pdf'))
# Fig 6b: Nodes colored by peptide composition
plot_graph(graph_fdr, peptide_pichart=True, max_neighborhood=50, layout=layout,
           graph_size=(200,200), edge_width=0.01, vertex_size=1.5, edge_color="#000000",
           output_fname=os.path.join(output_dir,'ppi_network_fdr0.01.peptide_color.max_adj50.pdf'))

(<GraphView object, undirected, with 1737 vertices and 20053 edges, 3 internal vertex properties, 1 internal edge property, edges filtered by (<EdgePropertyMap object with value type 'bool', for Graph 0x2e77d9250, at 0x2e77e86a0>, False), vertices filtered by (<VertexPropertyMap object with value type 'bool', for Graph 0x2e77d9250, at 0x2e77e8370>, False), at 0x2e77d9250>,
 <VertexPropertyMap object with value type 'vector<double>', for Graph 0x2e7089430, at 0x2e7665670>)

## Figure S7: PBD-associating sub-networks

In [34]:
for domain in domain_order:
    include_nodes = set([prot for prot in prots if domain in domain_data[prot]])
    print(domain, len(include_nodes))
    plot_graph(graph_fdr, included_neighborhood=include_nodes, domain_pichart=True, max_neighborhood=100,
               adjust_aspect=False, edge_width=0.01, vertex_size=2.5, edge_color="#000000", graph_size=(200,200),
               output_fname=os.path.join(output_dir, '%s_network_fdr0.01.domain_color.max_adj100.pdf' % domain))

SH2 110
PTB 30
Kinase_TK 90
PTP 37
WH1 11
WW 51
SH3 221
PDZ 154


## Network of PPIs with HSM/P predicted probability > 0.7

In [36]:
edges = [[str(p[0]), str(p[1]), float(p[2])] for p in ppis]
graph = load_graph(edges, domain_data, peptide_data)

In [37]:
_, layout = plot_graph(graph, domain_pichart=True, max_neighborhood=50, min_likelihood=0.7,
           adjust_aspect=False, graph_size=(200,200), edge_width=0.01, vertex_size=1.5, edge_color="#000000", seed=111,
           output_fname=os.path.join(output_dir,'ppi_network_p0.7.domain_color.max_adj50.pdf'))
plot_graph(graph, peptide_pichart=True, max_neighborhood=50, min_likelihood=0.7,
           adjust_aspect=False, layout=layout, graph_size=(200,200), edge_width=0.01, vertex_size=1.5, edge_color="#000000",
           output_fname=os.path.join(output_dir,'ppi_network_p0.7.peptide_color.max_adj50.pdf'))
for domain in domain_order:
    include_nodes = set([prot for prot in prots if domain in domain_data[prot]])
    print(domain, len(include_nodes))
    plot_graph(graph, included_neighborhood=include_nodes, domain_pichart=True, max_neighborhood=100,
               min_likelihood=0.7, adjust_aspect=False, graph_size=(200,200), edge_width=0.01, vertex_size=2.5, edge_color="#000000",
               output_fname=os.path.join(output_dir, '%s_network_p0.7.domain_color.max_adj100.pdf' % domain))

SH2 110
PTB 30
Kinase_TK 90
PTP 37
WH1 11
WW 51
SH3 221
PDZ 154
